# Vision Conversion

This example illustrates conversion from Vision excel export to power-grid-model input data.
They function in a similar way since both are [Tabular Converters](../converters/tabular_converter.md).
We can then calculate power-flow with it or convert to a different formats like PGM JSON.

## Vision conversion
![Vision example](data/vision/example.png)

### 1. Load the Vision data

To export the Vision file in Excel format, please follow the instructions given in the [Vision Manual](https://phasetophase.nl/pdf/VisionEN.pdf).

Define source and destination paths:

In [1]:
source_file = "data/vision/example.xlsx"
destination_file = "data/vision/sym_output.json"

Instantiate the converter, optionally with a source file path.
Then use `load_input_data()` to load the data and convert it to power-grid-model data.
The additional information that is not used in the powerflow calculation but may be useful to link the results to the source data is stored in `extra_info`.

In [2]:
%%capture cap --no-stderr
from power_grid_model_io.converters import VisionExcelConverter

converter = VisionExcelConverter(source_file=source_file)
input_data, extra_info = converter.load_input_data()

Let's investigate the data we have converted, for one of the components: `nodes`

In [3]:
import pandas as pd

# The node data is stored as a numpy structured array in input_data["node"]
display(input_data["node"])

# We can use pandas to display the data in a convenient tabular format
display(pd.DataFrame(input_data["node"]))

# Notice that the node names were not stored in the numpy array, as we don't need them for the calculations
display({i: extra_info[i]["Name"] for i in input_data["node"]["id"]})

array([(0, 400.), (1, 400.), (2, 400.), (3, 400.)],
      dtype={'names': ['id', 'u_rated'], 'formats': ['<i4', '<f8'], 'offsets': [0, 8], 'itemsize': 16, 'aligned': True})

,id,u_rated
0,0,400.0
1,1,400.0
2,2,400.0
3,3,400.0


{0: 'First Node', 1: 'Fourth Node', 2: 'Second Node', 3: 'Third Node'}

### 2. Validate the data
Before we run a power flow calculation, it is wise validate the data. The most basic method is to use `assert_valid_input_data()`, which will raise a `ValueError` when the data is invalid. For more details on data validation, please consult the [validation Example](https://github.com/alliander-opensource/power-grid-model/blob/main/docs/examples/Validation%20Examples.ipynb).

In [4]:
from power_grid_model import CalculationType
from power_grid_model.validation import assert_valid_input_data

assert_valid_input_data(input_data, calculation_type=CalculationType.power_flow, symmetric=True)

### 3. Run the calculation

Run powerflow calculation with the `input_data` and show the results for `nodes`.

In [5]:
from power_grid_model import PowerGridModel

pgm = PowerGridModel(input_data=input_data)
output_data = pgm.calculate_power_flow()

display(pd.DataFrame(output_data["node"]))

,id,energized,u_pu,u,u_angle
0,0,1,0.999991,399.996569,-0.000039
1,1,1,0.983232,393.292609,-0.003435
2,2,1,0.987662,395.064831,-0.003235
3,3,1,0.979773,391.909191,-0.006297


### Cross referencing objects
The converter has generated unique numerical IDs for all the components in the VisionExcel file, in fact for some special components like _Transformer loads_, multiple PGM components have been created, each with their own numerical ID. To find out which component belongs to which id, some helper functions have been defined:

In [6]:
print("PGM object #0:", converter.lookup_id(0))
print("PGM object #4:", converter.lookup_id(4))
print("PGM object #7:", converter.lookup_id(7))
print("PGM object #9:", converter.lookup_id(9))

print(
    "Node with Number=1:",
    converter.get_node_id(number=1)
)

print(
    "Cables with Number=6:",
    converter.get_branch_id(table="Cables", number=6)
)

print(
    "Source with Node.Number=1 and Subnumber=1:",
    converter.get_appliance_id(table="Sources", node_number=1, sub_number=1)
)

print(
    "Loads with Node.Number=4 and Subnumber=1:",
    converter.get_appliance_id(table="Loads", node_number=4, sub_number=1)
)

PGM object #0: {'table': 'Nodes', 'key': {'Number': 1}}
PGM object #4: {'table': 'Cables', 'key': {'Number': 6}}
PGM object #7: {'table': 'Sources', 'key': {'Node.Number': 1, 'Subnumber': 1}}
PGM object #9: {'table': 'Loads', 'key': {'Node.Number': 4, 'Subnumber': 1}}
Node with Number=1: 0
Cables with Number=6: 4
Source with Node.Number=1 and Subnumber=1: 7
Loads with Node.Number=4 and Subnumber=1: 9


### Saving the data as a JSON file
The data can be stored in a json file using the PgmJsonConverter. The file will be saved in the `destination_file` path supplied in the constructor.

In [7]:
from power_grid_model_io.converters import PgmJsonConverter

json_converter = PgmJsonConverter(destination_file=destination_file)
json_converter.save(data=output_data, extra_info=extra_info)

For debugging purposes, let's check the output JSON. Notice that the node names are added to the nodes data.

In [8]:
from pathlib import Path
from IPython.display import display, Markdown

with Path(destination_file).open() as json_file:
    display(Markdown(f"<pre style='max-height: 160px; overflow: scroll; white-space: pre'>{json_file.read()}</pre>"))

<pre style='max-height: 160px; overflow: scroll; white-space: pre'>{
  "line":
    [
      {"id": 4, "energized": 1, "loading": 0.15473990756817002, "p_from": 30133.178503728053, "q_from": 10050.721853721689, "i_from": 46.421809438989094, "s_from": 31765.16104977932, "p_to": -29999.9999999991, "q_to": -10000.000000000018, "i_to": 46.42197227045101, "s_to": 31622.776601682945, "id_reference": {"table": "Cables", "key": {"Number": 6}}},
      {"id": 5, "energized": 1, "loading": 0.2946347177166794, "p_from": 60482.835089115535, "q_from": 184.81468538821002, "i_from": 88.39041452675093, "s_from": 60483.11745342729, "p_to": -60000.00000000256, "q_to": -2.981555974335137e-10, "i_to": 88.3904153150038, "s_to": 60000.00000000256, "id_reference": {"table": "Cables", "key": {"Number": 5}}},
      {"id": 6, "energized": 1, "loading": 0.44422983507391905, "p_from": 91713.619010016, "q_from": 10656.105417735627, "i_from": 133.26889142389211, "s_from": 92330.60432266358, "p_to": -90616.01359284009, "q_to": -10235.536539109033, "i_to": 133.26895052217571, "s_to": 91192.25914353265, "id_reference": {"table": "Cables", "key": {"Number": 4}}}
    ],
  "node":
    [
      {"id": 0, "energized": 1, "u_pu": 0.999991421302051, "u": 399.9965685208204, "u_angle": -3.921691889422987e-05, "id_reference": {"table": "Nodes", "key": {"Number": 1}}, "ID": 101, "Name": "First Node"},
      {"id": 1, "energized": 1, "u_pu": 0.983231522194849, "u": 393.2926088779396, "u_angle": -0.003434581394274798, "id_reference": {"table": "Nodes", "key": {"Number": 4}}, "ID": 104, "Name": "Fourth Node"},
      {"id": 2, "energized": 1, "u_pu": 0.9876620765421245, "u": 395.0648306168498, "u_angle": -0.003235042286685305, "id_reference": {"table": "Nodes", "key": {"Number": 2}}, "ID": 102, "Name": "Second Node"},
      {"id": 3, "energized": 1, "u_pu": 0.9797729773054616, "u": 391.9091909221847, "u_angle": -0.006296501518419363, "id_reference": {"table": "Nodes", "key": {"Number": 3}}, "ID": 103, "Name": "Third Node"}
    ],
  "source":
    [
      {"id": 7, "energized": 1, "p": 91713.61901002181, "q": 10656.105417760706, "i": 133.26889142390465, "s": 92330.60432267224, "pf": 0.9933176510954675, "id_reference": {"table": "Sources", "key": {"Node.Number": 1, "Subnumber": 1}}, "Name": "Grid"}
    ],
  "sym_load":
    [
      {"id": 8, "energized": 1, "p": 60000.0, "q": -0.0, "i": 88.39041531500003, "s": 60000.0, "pf": 1.0, "id_reference": {"table": "Loads", "key": {"Node.Number": 3, "Subnumber": 1}}},
      {"id": 9, "energized": 1, "p": 30000.0, "q": 10000.0, "i": 46.421972270452244, "s": 31622.776601683792, "pf": 0.9486832980505139, "id_reference": {"table": "Loads", "key": {"Node.Number": 4, "Subnumber": 1}}}
    ]
}
</pre>

## Other languages
Currently Dutch is the only extra language that is supported for conversion.

In [9]:
%%capture cap --no-stderr

converter = VisionExcelConverter(source_file=source_file, language="nl")

## Summary

In [10]:
%%capture cap --no-stderr

from power_grid_model import PowerGridModel, CalculationType
from power_grid_model.validation import assert_valid_input_data
from power_grid_model_io.converters import VisionExcelConverter, PgmJsonConverter

source_file = "data/vision/example.xlsx"
destination_file = "data/vision/sym_output.json"

converter = VisionExcelConverter(source_file=source_file)
input_data, extra_info = converter.load_input_data()
assert_valid_input_data(input_data, calculation_type=CalculationType.power_flow, symmetric=True)
pgm = PowerGridModel(input_data=input_data)
output_data = pgm.calculate_power_flow()
json_converter = PgmJsonConverter(destination_file=destination_file)
json_converter.save(data=output_data, extra_info=extra_info)